In [2]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None


################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  return b''


################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  return b''


################################################################################
# Problem 3 SOLUTION
################################################################################

def problem3(cnetid):
  return b''


################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''


################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [3]:
ctext = bytearray(make_query('fivea', 'ruolinzheng', ''))

In [4]:
make_query('fiveb', 'ruolinzheng', ctext)

b'true'

In [5]:
make_query('fiveb', 'ruolinzheng', ctext[:-16])

b'false'

In [6]:
# maybe last byte of msg is already 0x01?
ctext[-17] = 1 ^ 2
# try to make msg[-2] 0x01
idx = -18
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'ruolinzheng', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(flag_byte, bytes([flag_byte]))
# nope

In [7]:
ctext = bytearray(make_query('fivea', 'davidcash', ''))
# try to make msg[-1] 0x01
idx = 31
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'davidcash', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(i, flag_byte, bytes([flag_byte]))

0 75 b'K'
13 70 b'F'


In [8]:
ctext[31] = 70 ^ 2
idx = 30
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'davidcash', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(i, flag_byte, bytes([flag_byte]))

10 213 b'\xd5'


In [9]:
ctext[31] = 70 ^ 3
ctext[30] = 213 ^ 2
idx = 29
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'davidcash', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(i, flag_byte, bytes([flag_byte]))

11 109 b'm'


In [10]:
ctext[31] = 70 ^ 4
ctext[30] = 213 ^ 3
ctext[29] = 109 ^ 2
idx = 28
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'davidcash', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(i, flag_byte, bytes([flag_byte]))

8 73 b'I'


In [11]:
ctext[31] = 70 ^ 5
ctext[30] = 213 ^ 4
ctext[29] = 109 ^ 3
ctext[28] = 73 ^ 2
idx = 27
for i in range(256):
  ctext[idx] ^= i
  resp = make_query('fiveb', 'davidcash', ctext)
  ctext[idx] ^= i # reset
  if resp == b'true':
    flag_byte = ctext[idx] ^ i ^ 1
    print(i, flag_byte, bytes([flag_byte]))

9 195 b'\xc3'


In [23]:
def get_flag_byte(ctext, idx):
  flag_byte = None
  for i in range(1, 256):
    ctext[idx] ^= i
    resp = make_query('fiveb', 'davidcash', ctext)
    ctext[idx] ^= i # reset
    if resp == b'true':
      flag_byte = ctext[idx] ^ i ^ 1
#       print(i, flag_byte, bytes([flag_byte]))
      break
  return flag_byte

In [13]:
print([c for c in ctext[27:32]])

[203, 75, 110, 209, 67]


In [14]:
recovered = bytearray(28) + bytearray([73, 109, 213, 70])
for i in range(4):
  print(recovered[28 + i], ((i + 2) % 256), 
        recovered[28 + i] ^ ((i + 2) % 256))
print([c for c in ctext[28:32]])

73 2 75
109 3 110
213 4 209
70 5 67
[75, 110, 209, 67]


In [28]:
def get_flag_byte(cnetid, ctext, idx):
  flag_byte = None
  for i in range(1, 256):
    ctext[idx] ^= i
    resp = make_query('fiveb', cnetid, ctext)
    ctext[idx] ^= i # reset
    if resp == b'true':
      flag_byte = ctext[idx] ^ i ^ 1
#       print(i, flag_byte, bytes([flag_byte]))
      break
  return flag_byte

def set_ctext(ctext, recovered, start, end):
  for i in range(end - start):
    offset = start + i
    byte = (i + 2) % 256 # wrap around
#     print(recovered[offset] ^ byte)
    ctext[offset] = recovered[offset] ^ byte

In [21]:
recovered = bytearray(28) + bytearray([73, 109, 213, 70])
set_ctext(ctext, recovered, 28, 32)
print([c for c in ctext[28:32]])

[75, 110, 209, 67]


In [25]:
ctext = bytearray(make_query('fivea', 'davidcash', ''))
len_msg = len(ctext) - 16
recovered = bytearray(len_msg)
for idx in range(len_msg - 1, -1, -1):
  flag_byte = get_flag_byte(ctext, idx)
  print(idx, flag_byte, bytes([flag_byte]))
  recovered[idx] = flag_byte
  set_ctext(ctext, recovered, idx, len_msg)

31 70 b'F'
30 213 b'\xd5'
29 109 b'm'
28 73 b'I'
27 195 b'\xc3'
26 247 b'\xf7'
25 152 b'\x98'
24 222 b'\xde'
23 175 b'\xaf'
22 6 b'\x06'
21 18 b'\x12'


TypeError: 'NoneType' object cannot be interpreted as an integer

In [26]:
ctext = bytearray(make_query('fivea', 'davidcash', ''))
ctext = ctext[:-16]
len_msg = len(ctext) - 16
recovered = bytearray(len_msg)
for idx in range(len_msg - 1, -1, -1):
  flag_byte = get_flag_byte(ctext, idx)
  print(idx, flag_byte, bytes([flag_byte]))
  recovered[idx] = flag_byte
  set_ctext(ctext, recovered, idx, len_msg)

15 88 b'X'
14 67 b'C'
13 67 b'C'
12 84 b'T'
11 69 b'E'
10 17 b'\x11'
9 66 b'B'
8 88 b'X'
7 17 b'\x11'
6 112 b'p'
5 98 b'b'
4 127 b'\x7f'
3 17 b'\x11'
2 84 b'T'
1 89 b'Y'
0 101 b'e'


In [27]:
recovered

bytearray(b'eYT\x11\x7fbp\x11XB\x11ETCCX')

In [29]:
cnetid = 'ruolinzheng'
ctext = bytearray(make_query('fivea', cnetid, ''))
ctext = ctext[:-16]
len_msg = len(ctext) - 16
recovered = bytearray(len_msg)
for idx in range(len_msg - 1, -1, -1):
  flag_byte = get_flag_byte(cnetid, ctext, idx)
  print(idx, flag_byte, bytes([flag_byte]))
  recovered[idx] = flag_byte
  set_ctext(ctext, recovered, idx, len_msg)

15 117 b'u'
14 17 b'\x11'
13 66 b'B'
12 95 b'_'
11 80 b'P'
10 67 b'C'
9 84 b'T'
8 69 b'E'
7 84 b'T'
6 103 b'g'
5 17 b'\x11'
4 72 b'H'
3 65 b'A'
2 65 b'A'
1 80 b'P'
0 121 b'y'
